In [25]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
# 수치 미분의 경우, 단순하고 구현하기엔 쉽지만, 계산 시간이 오래 걸리는 단점이 있음.
# 오차역전파법은 Chain Rule 을 이용하여 위의 문제를 해결함.
# 중간까지 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산하는 것이 가능함.

# 덧셈 노드의 역전파는 1을 곱하기만 할 뿐, 역전파 값을 그대로 하류로 보냄.
# 곱셈 노드의 역전파는 순전파 때의 입력 신호들을 '서로 바꾼 값'을 역전파 값과 곱하여 하류로 보냄.

# '차분을 이용한 수치 미분'과 '역전파'의 차이는,
# 수치 미분의 경우에 모든 값들에 대하여 직접 미분을 계산하는 반면, 역전파는 순전파에서 이용했던 입력 신호를 가져다 계산만 한다는 점에 있다.

### 단순한 계층 구현하기

In [3]:
# 곱셈 계층

class MulLayer:
    
    def __init__(self):
        
        # 순전파 시의 입력 값을 유지하기 위해 변수 할당
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        
        self.x = x
        self.y = y
        out = x * y
        
        return out
        
    def backward(self, dout):
        
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [4]:
# 순전파

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward( apple_price, tax )

print( int(price) )

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, '\t', dapple_num, '\t', dtax)

220
2.2 	 110.00000000000001 	 200


In [5]:
# 덧셈 계층

class AddLayer():
    
    def __init__(self):
        pass
    
    def forward(self, x, y):
        
        out = x + y
        
        return out
    
    def backward(self, dout):
        
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [6]:
apple_price = 100
apple_num = 2

orange_price = 150
orange_num = 3

tax = 1.1

# 순전파

apple = MulLayer()
orange = MulLayer()

mul_apple = apple.forward(apple_price, apple_num)
mul_orange = orange.forward(orange_price, orange_num)

shopping = AddLayer()
total_price = shopping.forward(mul_apple, mul_orange)

plus_tax = MulLayer()
price = plus_tax.forward(total_price, tax)

print(price)

# 역전파

dprice = 1

dtotal_price, dtax = plus_tax.backward(1)
dmul_apple, dmul_orange = shopping.backward(dtotal_price)
dapple_price, dapple_num = apple.backward(dmul_apple)
dorange_price, dorange_num = orange.backward(dmul_orange)

print(dapple_price, dapple_num, dorange_price, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


### 활성화 함수 계층

In [7]:
class ReLU():
    
    def __init__(self):
        
        self.mask = None
    
    def forward(self, x):
        
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [22]:
class Sigmoid:
    
    def __init(self):
        
        self.out = None
        pass
    
    def forward(self, x):
        
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        
        dx = dout * self.out * (1.0 - self.out)
        
        return dx

### Affine / Softmax 계층 구현

In [23]:
# Affine Transformation
# 기저벡터의 linear transformation을 통해 input 데이터의 기저벡터를 변환하여 표현

class Affine:
    
    def __init__(self, W, b):
        
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
        
    def forward(self, x):
        
        self.x = x
        out = np.dot(x, self. W) + self.b
        
        return out
    
    def backward(self, dout):
        
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

In [24]:
# Softmax_with_Loss
# 신경망 추론과 다르게 학습에서 Softmax 층이 필요한 이유는, 손실함수를 통해 출력되는 지표값을 참고하여 backprop이 이루어지기 때문

class SoftmaxWithLoss:
    
    def __init(self):
        
        self.loss = None
        self.y = None
        self.t = None    
    
    
    def forward(self, x, t):
        
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    
    def backward(self, dout=1):
        
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [ ]:
class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        
    
    
    
    def predict

    
    
    def loss
    
    
    
    def accuracy
    
    
    
    def gradient
    
    
        